
# Notion-Export robust laden (CSV/XLSX) & Wort-Statistiken je Zeile

Dieses Notebook ist **robust** gegen den Fall, dass Excel **alles in Spalte A** anzeigt:
- Auto-Delimiter-Erkennung (`,`, `;`, `\t`)
- Fallback, falls die Datei zunächst als **eine Spalte** eingelesen wird
- Unterstützt auch **.xlsx** (Excel-Datei)
- Erkennt Spalten „Output 1 … Output 10“ bzw. „Ausgabe 1 …“
- Berechnet je Zeile **n**, **Mean**, **Median** der Wortanzahlen über alle Outputs


In [14]:

import re
import csv
import pandas as pd
from statistics import mean, median
from pathlib import Path

WORD_SPLIT = re.compile(r"\s+")

def count_words(val) -> int:
    if isinstance(val, str):
        s = val.strip()
        if not s:
            return 0
        tokens = [t for t in WORD_SPLIT.split(s) if t]
        return len(tokens)
    return 0

def detect_output_columns(columns):
    outs = []
    pattern = re.compile(r"^(output|ausgabe)\s*\d+$", re.IGNORECASE)
    for c in columns:
        c_norm = str(c).strip().replace("\u00A0", " ")
        if pattern.match(c_norm):
            outs.append(c)
    if not outs:
        pattern2 = re.compile(r"^(output|ausgabe)\d+$", re.IGNORECASE)
        for c in columns:
            c_norm = str(c).strip()
            if pattern2.match(c_norm):
                outs.append(c)
    def _num(c):
        m = re.search(r"(\d+)$", str(c))
        return int(m.group(1)) if m else 9999
    return sorted(outs, key=_num)

def row_stats(counts):
    clean = [int(x) for x in counts if isinstance(x, (int, float)) and x is not None]
    clean = [c for c in clean if c >= 0]
    if not clean:
        return {"n": 0, "mean": 0.0, "median": 0.0}
    return {"n": len(clean), "mean": float(mean(clean)), "median": float(median(clean))}

def read_table(path_str: str):
    path = Path(path_str)
    if not path.exists():
        raise FileNotFoundError(f"Datei nicht gefunden: {path}")
    ext = path.suffix.lower()

    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)

    # try sniffing delimiter
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        sample = f.read(4096)
    try:
        dialect = csv.Sniffer().sniff(sample, delimiters=[",",";","\t","|"])
        sep = dialect.delimiter
    except Exception:
        sep = ","

    df = pd.read_csv(path, sep=sep, encoding="utf-8", engine="python")

    if df.shape[1] == 1:
        col = df.columns[0]
        head = str(col)
        # Try common candidates based on header
        for cand in [",",";","\t","|"]:
            if cand in head:
                df = pd.read_csv(path, sep=cand, encoding="utf-8", engine="python")
                break

    return df


In [15]:

# === Konfiguration ===
CSV_PATH = "C:\\Users\\Marius\\Desktop\\Dev\\CodePilot\\app\\evaluation\\data\\Persona3.csv"   # <--- hier anpassen
OUTPUT_STATS_CSV = "notion_output_word_stats_Persona3.csv"
PREVIEW_ROWS = 10


In [16]:

# === Laden & Prüfen ===
df = read_table(CSV_PATH)
print(f"Geladene Form: {df.shape[0]} Zeilen, {df.shape[1]} Spalten")
display(df.head(3))


Geladene Form: 4 Zeilen, 13 Spalten


,Wo,Prompt,Input,Output 1,Output 2,Output 3,Output 4,Output 5,Output 6,Output 7,Output 8,Output 9,Output 10
0,Allgemeiner\nTemperature: Default\n,Kontext: {{$description}}\nLernpräferenzen: {{...,public int[] IncrementArray(int[] nums) {\nint...,1. Schau dir die Schleifenbedingung an: „for (...,"Überlege, was passiert, wenn i den Wert nums.L...","Bevor wir in den Code eintauchen, fass bitte i...",Deine Schleifenbedingung `i <= nums.Length` fü...,Deine Schleifenbedingung i <= nums.Length führ...,1. Schau dir die Schleifenbedingung an – mit `...,Deine Schleifenbedingung i <= nums.Length läuf...,1. Deine Schleifenbedingung `i <= nums.Length`...,Deine Schleifenbedingung „i <= nums.Length“ fü...,Schau dir die Schleifenbedingung genau an: Bei...
1,Plugin KH\nTemperature: Default,KH ( Knowledge on how to proceed )\n\nKontext:...,public int[] IncrementArray(int[] nums) {\nint...,"Überprüfe, ob deine Schleifenbedingung wirklic...",Überprüfe deine Schleifenbedingung: Mit `i <= ...,"Überprüfe, ob deine Schleifenbedingung die ric...",Überprüfe deine Schleifenbedingung: Stelle sic...,Überprüfe deine Schleifenbedingung: Stelle sic...,"Überprüfe, ob deine Schleifenbedingung i <= nu...","Überprüfe, ob deine Abbruchbedingung i <= nums...","Überprüfe, ob deine Schleifenbedingung korrekt...",Überprüfe deine Schleifenbedingung – statt `i ...,Überprüfe deine Schleifenbedingung im forKopf ...
2,Plugin KM \nTemperature: Default,KM (Knowledge about Mistakes) Prompt\n\nKontex...,public int[] IncrementArray(int[] nums) {\nint...,Fehlerübersicht:\n- In der Schleifenbedingung ...,In Zeile 3 (`for (int i = 0; i <= nums.Length;...,1. In der for-Schleife (`for (int i = 0; i <= ...,1. In der for-Schleifen­bedingung „i <= nums.L...,Fehleranalyse:\n1. Schleifenbedingung (Zeile 3...,Fehlerübersicht:\n- Schleifenbedingung in Zeil...,In der Methode fehlt eine `return`-Anweisung f...,In der Schleifen­bedingung (`for (int i = 0; i...,In der Schleifenbedingung `for (int i = 0; i <...,In der for-Schleife (Zeile 3) lautet die Bedin...


In [17]:

# === Output-Spalten finden & Wortanzahlen berechnen ===
output_cols = detect_output_columns(df.columns)
if not output_cols:
    output_cols = [c for c in df.columns if str(c).strip().lower().startswith("output")]
if not output_cols:
    raise ValueError("Keine Output-Spalten gefunden. Benenne Spalten z. B. 'Output 1' ... 'Output 10'.")

print("Gefundene Output-Spalten:", output_cols)

for col in output_cols:
    df[f"{col}_words"] = df[col].apply(count_words)

word_cols = [f"{c}_words" for c in output_cols]

stats_n, stats_mean, stats_median = [], [], []
for _, row in df.iterrows():
    counts = [row[c] for c in word_cols]
    st = row_stats(counts)
    stats_n.append(st["n"])
    stats_mean.append(st["mean"])
    stats_median.append(st["median"])

df["outputs_n"] = stats_n
df["outputs_mean"] = stats_mean
df["outputs_median"] = stats_median

display(df.head(PREVIEW_ROWS)[word_cols + ["outputs_n","outputs_mean","outputs_median"]])


Gefundene Output-Spalten: ['Output 1', 'Output 2', 'Output 3', 'Output 4', 'Output 5', 'Output 6', 'Output 7', 'Output 8', 'Output 9', 'Output 10']


,Output 1_words,Output 2_words,Output 3_words,Output 4_words,Output 5_words,Output 6_words,Output 7_words,Output 8_words,Output 9_words,Output 10_words,outputs_n,outputs_mean,outputs_median
0,71,58,35,51,56,67,64,61,66,71,10,60.0,62.5
1,22,30,22,30,24,21,23,36,24,23,10,25.5,23.5
2,90,62,56,35,94,53,51,48,48,32,10,56.9,52.0
3,5,5,5,5,5,5,5,5,5,5,10,5.0,5.0


In [18]:

# === Speichern ===
df_out = df[word_cols + ["outputs_n","outputs_mean","outputs_median"]].copy()
df_out.to_csv(OUTPUT_STATS_CSV, index=False, encoding="utf-8")
print(f"Ergebnisse gespeichert in: {OUTPUT_STATS_CSV}")


Ergebnisse gespeichert in: notion_output_word_stats_Persona3.csv


In [19]:
from pathlib import Path
import json

# 1) Pfad wählen: gleiche Basis wie CSV, aber .json
JSON_PATH = Path(OUTPUT_STATS_CSV).with_suffix(".json")
JSONL_PATH = Path(OUTPUT_STATS_CSV).with_suffix(".jsonl")

# 2) Records pro Zeile bauen
records = []
for idx, row in df.iterrows():
    record = {
        "row": int(idx) + 1,  # oder nimm eine ID-Spalte, falls vorhanden
        "outputs": {col.replace("_words", ""): int(row[col]) for col in word_cols},
        "n": int(row["outputs_n"]),
        "mean": float(row["outputs_mean"]),
        "median": float(row["outputs_median"]),
    }
    records.append(record)

# 3a) Schön formatiertes JSON (Array)
with open(JSON_PATH, "w", encoding="utf-8") as f:
    json.dump(records, f, ensure_ascii=False, indent=2)
print(f"JSON gespeichert in: {JSON_PATH}")

# 3b) Optional: JSONL (eine Zeile = ein Objekt)
with open(JSONL_PATH, "w", encoding="utf-8") as f:
    for rec in records:
        f.write(json.dumps(rec, ensure_ascii=False) + "\n")
print(f"JSONL gespeichert in: {JSONL_PATH}")

JSON gespeichert in: notion_output_word_stats_Persona3.json
JSONL gespeichert in: notion_output_word_stats_Persona3.jsonl
